In [1]:
from pprint import pprint 
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

In [2]:
from blogging_app.models import Post, PostCategory, Like, Comment, User

In [4]:
from enum import Enum

import json 

from rest_framework.test import APIRequestFactory
from rest_framework.request import Request
from rest_framework.parsers import JSONParser
from rest_framework.response import Response
from rest_framework import status

factory = APIRequestFactory()

class HTTP_METHODS(Enum):
    GET = "GET"
    POST = "POST"
    PATCH = "PATCH"
    PUT = "PUT"
    DELETE = "DELETE"


def get_drf_request(method, path, data):
    
    factory_methods = {
        "GET": factory.get,
        "POST": factory.post,
        "PATCH": factory.patch,
        "PUT": factory.put,
        "DELETE": factory.delete
    }
    
    method = factory_methods[method]
    
    wsgi_request_instance = method(
        path=path, 
        data=data,
        format='json'
    )

    print(wsgi_request_instance)

    drf_request_instance = Request(
        request=wsgi_request_instance,
        parsers=[JSONParser()],
    )

    print(drf_request_instance)

    print(drf_request_instance.__dict__,"\n\n")

    print("JSON data sent in the request:\n", drf_request_instance.data, "\n\n")
    print("Query Params sent in the request:\n",drf_request_instance.query_params, "\n\n")

    return drf_request_instance

# Deserialization and validation

In [5]:
def create_post_category(request):
    """
    This function:
        1. Extracts the data coming in the request.
        2. Validates it. If the validation fails returns a "400"-"Bad Request" Response.
        3. Serializes the data if the request data passed the validation & creates a new PostCategory instance
        4. Saves the instance in the DB  
        5. Prepares the response data and returns a "201" - "Created" Response
    """

    # 1. Extracting the data coming in the request
    request_data = request.data

    # 2. Validates request_data If the validation fails returns a "400"-"Bad Request" Response.
    if "name" not in request_data:
        return Response(
            data={
                "message": f"Name is a requried field"
            },
            status=status.HTTP_400_BAD_REQUEST
        )
    elif any( char.isdigit() for char in request_data.get("name") ) :
        return Response(
            data={
                "message": "Post category names are not allowed to have numbers"
            },
            status=status.HTTP_400_BAD_REQUEST
        )
    
    # 3. Serializes the data if the request data passed the validation & creates a new PostCategory instance
    new_post_category_instance = PostCategory(name=request_data["name"])

    # 4. Saves the instance in the DB 
    new_post_category_instance.save()

    # 5. Prepares the response data and returns a "201" - "Created" Response
    response_data = {
        "name": new_post_category_instance.name,
        "id": new_post_category_instance.id
    }
    
    return Response(
        data={
            "message": "New tag record created",
            "data": response_data,
        },
        status=status.HTTP_201_CREATED
    )


In [6]:
def update_post_category(request, post_category_id):
    """
    This function:
        0. Checks if the requested post category instance exists
        1. Extracts the data coming in the request.
        2. Validates it. If the validation fails returns a "400"-"Bad Request" Response.
        3. DeSerializes the data if the request data passed the validation & updates the requested PostCategory instance
        4. Prepares the response data and returns a "200" - "OK" Response
    """

    # 0. Checks if the requested post category instance exists
    try:
        post_category_instance = PostCategory.objects.get(id=post_category_id)
    except PostCategory.DoesNotExist:
        return Response(
            data={
                "message": "Requested post category record does not exists"
            },
            status=status.HTTP_404_NOT_FOUND
        )

    # 1. Extracting the data coming in the request
    request_data = request.data

    # 2. Validates request_data If the validation fails returns a "400"-"Bad Request" Response.
    if "name" not in request_data:
        return Response(
            data={
                "message": f"Name is a requried field"
            },
            status=status.HTTP_400_BAD_REQUEST
        )
    elif any( char.isdigit() for char in request_data.get("name") ) :
        return Response(
            data={
                "message": "Post category names are not allowed to have numbers"
            },
            status=status.HTTP_400_BAD_REQUEST
        )
    
    # 3. DeSerializes the data if the request data passed the validation & updates the requested PostCategory instance
    post_category_instance.name=request_data["name"]
    post_category_instance.save()

    # 4. Prepares the response data and returns a "200" - "OK" Response
    response_data = {
        "name": post_category_instance.name,
        "id": post_category_instance.id
    }
    
    return Response(
        data={
            "message": "New post category record created",
            "data": response_data,
        },
        status=status.HTTP_200_CREATED
    )

    


In [8]:
print("-"*40,"Request","-"*40)
post_request = get_drf_request(
    method=HTTP_METHODS.POST.value,
    path="/post-category",
    data={
        "name": "System Design 123" 
    }
)
print("-"*100)


print("\n\n","="*40,"Response","="*40)

response = create_post_category(post_request)
pprint(response.status_code)
pprint(response.data)
print("="*100)


---------------------------------------- Request ----------------------------------------
<WSGIRequest: POST '/post-category'>
<rest_framework.request.Request: POST '/post-category'>
{'_request': <WSGIRequest: POST '/post-category'>, 'parsers': [<rest_framework.parsers.JSONParser object at 0x70f682d9b2b0>], 'authenticators': (), 'negotiator': <rest_framework.negotiation.DefaultContentNegotiation object at 0x70f68383ba30>, 'parser_context': {'request': <rest_framework.request.Request: POST '/post-category'>, 'encoding': 'utf-8'}, '_data': <class 'rest_framework.request.Empty'>, '_files': <class 'rest_framework.request.Empty'>, '_full_data': <class 'rest_framework.request.Empty'>, '_content_type': <class 'rest_framework.request.Empty'>, '_stream': <class 'rest_framework.request.Empty'>} 


JSON data sent in the request:
 {'name': 'System Design 123'} 


Query Params sent in the request:
 <QueryDict: {}> 


----------------------------------------------------------------------------------

In [9]:
print("-"*40,"Request","-"*40)
post_request = get_drf_request(
    method=HTTP_METHODS.POST.value,
    path="/post-category",
    data={
        "name": "Low-Level System Design" 
    }
)
print("-"*100)


print("\n\n","="*40,"Response","="*40)

response = create_post_category(post_request)
pprint(response.status_code)
pprint(response.data)
print("="*100)

---------------------------------------- Request ----------------------------------------
<WSGIRequest: POST '/post-category'>
<rest_framework.request.Request: POST '/post-category'>
{'_request': <WSGIRequest: POST '/post-category'>, 'parsers': [<rest_framework.parsers.JSONParser object at 0x70f68383a320>], 'authenticators': (), 'negotiator': <rest_framework.negotiation.DefaultContentNegotiation object at 0x70f68383b5e0>, 'parser_context': {'request': <rest_framework.request.Request: POST '/post-category'>, 'encoding': 'utf-8'}, '_data': <class 'rest_framework.request.Empty'>, '_files': <class 'rest_framework.request.Empty'>, '_full_data': <class 'rest_framework.request.Empty'>, '_content_type': <class 'rest_framework.request.Empty'>, '_stream': <class 'rest_framework.request.Empty'>} 


JSON data sent in the request:
 {'name': 'Low-Level System Design'} 


Query Params sent in the request:
 <QueryDict: {}> 


----------------------------------------------------------------------------

In [25]:
from rest_framework.serializers import ModelSerializer, ValidationError, CharField

class PostCategoryCreateUpdateSerializer(ModelSerializer):
    """
    This serializer is responsible for the serialization &
    de-serialization for PostCategory model recrods.
    """

    name = CharField(required=True)

    def validate_name(self, value):
        print("validate_name method of PostCategorySerializer is called!")
        if any( char.isdigit() for char in value ) :
            print("Numbers found")
            validation_error = ValidationError(
                detail="Post category names are not allowed to have numbers",
            )
            raise validation_error
        return value

    class Meta:
        model = PostCategory
        fields = ( "name", )

In [63]:
def create_post_category(request):
    """
    This function:
        1. Extracts the data coming in the request.
        2. Validates it. If the validation fails returns a "400"-"Bad Request" Response.
        3. DeSerializes the data if the request data passed the validation & creates a new PostCategory instance & saves the instance in the DB
        5. Prepares the response data and returns a "201" - "Created" Response
    """

    # 1. Extracting the data coming in the request
    request_data = request.data
    serializer_instance = PostCategoryCreateUpdateSerializer(data=request.data)

    # 2. Validates request_data If the validation fails returns a "400"-"Bad Request" Response.
    try:
        serializer_instance.is_valid(raise_exception=True)
    except ValidationError as e:
        return Response(
            data={
                "message": e.detail[list(e.detail.keys())[0]][0]
            },
            status=status.HTTP_400_BAD_REQUEST
        )
    
    # 3. DeSerializes the data if the request data passed the validation, creates a new PostCategory instance & saves the instance in the DB
    new_post_category_instance = serializer_instance.create(validated_data=serializer_instance.validated_data)

    # 4. Prepares the response data and returns a "201" - "Created" Response
    response_data = {
        "name": new_post_category_instance.name,
        "id": new_post_category_instance.id
    }
    
    return Response(
        data={
            "message": "New post category record created",
            "data": response_data,
        },
        status=status.HTTP_201_CREATED
    )

In [61]:
print("-"*40,"Request","-"*40)
post_request = get_drf_request(
    method=HTTP_METHODS.POST.value,
    path="/post-category",
    data={
        "name": "System Design 123" 
    }
)
print("-"*100)


print("\n\n","="*40,"Response","="*40)

response = create_post_category(post_request)
pprint(response.status_code)
pprint(response.data)
print("="*100)

---------------------------------------- Request ----------------------------------------
<WSGIRequest: POST '/post-category'>
<rest_framework.request.Request: POST '/post-category'>
{'_request': <WSGIRequest: POST '/post-category'>, 'parsers': [<rest_framework.parsers.JSONParser object at 0x7fecb2717ee0>], 'authenticators': (), 'negotiator': <rest_framework.negotiation.DefaultContentNegotiation object at 0x7fecb2715720>, 'parser_context': {'request': <rest_framework.request.Request: POST '/post-category'>, 'encoding': 'utf-8'}, '_data': <class 'rest_framework.request.Empty'>, '_files': <class 'rest_framework.request.Empty'>, '_full_data': <class 'rest_framework.request.Empty'>, '_content_type': <class 'rest_framework.request.Empty'>, '_stream': <class 'rest_framework.request.Empty'>} 


JSON data sent in the request:
 {'name': 'System Design 123'} 


Query Params sent in the request:
 <QueryDict: {}> 


----------------------------------------------------------------------------------

In [64]:
print("-"*40,"Request","-"*40)
post_request = get_drf_request(
    method=HTTP_METHODS.POST.value,
    path="/post-category",
    data={
        "name": "Microservice Design" 
    }
)
print("-"*100)


print("\n\n","="*40,"Response","="*40)

response = create_post_category(post_request)
pprint(response.status_code)
pprint(response.data)
print("="*100)

---------------------------------------- Request ----------------------------------------
<WSGIRequest: POST '/post-category'>
<rest_framework.request.Request: POST '/post-category'>
{'_request': <WSGIRequest: POST '/post-category'>, 'parsers': [<rest_framework.parsers.JSONParser object at 0x7fecb28899c0>], 'authenticators': (), 'negotiator': <rest_framework.negotiation.DefaultContentNegotiation object at 0x7fecb2888f70>, 'parser_context': {'request': <rest_framework.request.Request: POST '/post-category'>, 'encoding': 'utf-8'}, '_data': <class 'rest_framework.request.Empty'>, '_files': <class 'rest_framework.request.Empty'>, '_full_data': <class 'rest_framework.request.Empty'>, '_content_type': <class 'rest_framework.request.Empty'>, '_stream': <class 'rest_framework.request.Empty'>} 


JSON data sent in the request:
 {'name': 'Microservice Design'} 


Query Params sent in the request:
 <QueryDict: {}> 


--------------------------------------------------------------------------------

In [72]:
def update_post_category(request, post_category_id):
    """
    This function:
        1. Extracts the data coming in the request.
        2. Validates it. If the validation fails returns a "400"-"Bad Request" Response.
        3. DeSerializes the data if the request data passed the validation & updates the requested Post Category instance
        4. Prepares the response data and returns a "200" - "OK" Response
    """

    # 0. Checks if the requested post category instance exists
    try:
        post_category_instance = PostCategory.objects.get(id=post_category_id)
    except PostCategory.DoesNotExist:
        return Response(
            data={
                "message": "Requested post category record does not exists"
            },
            status=status.HTTP_404_NOT_FOUND
        )
    
    # 1. Extracting the data coming in the request
    request_data = request.data
    serializer_instance = PostCategoryCreateUpdateSerializer(data=request.data)

    # 2. Validates request_data If the validation fails returns a "400"-"Bad Request" Response.
    try:
        serializer_instance.is_valid(raise_exception=True)
    except ValidationError as e:
        return Response(
            data={
                "message": e.detail[list(e.detail.keys())[0]][0]
            },
            status=status.HTTP_400_BAD_REQUEST
        )
    
    # 3. DeSerializes the data if the request data passed the validation & updates the requested Post Category instance
    updated_post_category_instance = serializer_instance.update(instance=post_category_instance, validated_data=serializer_instance.validated_data)

    # 4. Prepares the response data and returns a "200" - "OK" Response
    response_data = {
        "name": updated_post_category_instance.name,
        "id": updated_post_category_instance.id
    }
    
    return Response(
        data={
            "message": "Post category record updated",
            "data": response_data,
        },
        status=status.HTTP_200_OK
    )

In [66]:
print("-"*40,"Request","-"*40)
patch_request = get_drf_request(
    method=HTTP_METHODS.PATCH.value,
    path="/post-category/6",
    data={
        "name": "Microservice Architecture 123" 
    }
)
print("-"*100)


print("\n\n","="*40,"Response","="*40)

response = update_post_category(patch_request, post_category_id=6)
pprint(response.status_code)
pprint(response.data)
print("="*100)

---------------------------------------- Request ----------------------------------------
<WSGIRequest: PATCH '/post-category/6'>
<rest_framework.request.Request: PATCH '/post-category/6'>
{'_request': <WSGIRequest: PATCH '/post-category/6'>, 'parsers': [<rest_framework.parsers.JSONParser object at 0x7fecb27cd990>], 'authenticators': (), 'negotiator': <rest_framework.negotiation.DefaultContentNegotiation object at 0x7fecb27cf1c0>, 'parser_context': {'request': <rest_framework.request.Request: PATCH '/post-category/6'>, 'encoding': 'utf-8'}, '_data': <class 'rest_framework.request.Empty'>, '_files': <class 'rest_framework.request.Empty'>, '_full_data': <class 'rest_framework.request.Empty'>, '_content_type': <class 'rest_framework.request.Empty'>, '_stream': <class 'rest_framework.request.Empty'>} 


JSON data sent in the request:
 {'name': 'Microservice Architecture 123'} 


Query Params sent in the request:
 <QueryDict: {}> 


----------------------------------------------------------

In [73]:
print("-"*40,"Request","-"*40)
patch_request = get_drf_request(
    method=HTTP_METHODS.PATCH.value,
    path="/post-category/6",
    data={
        "name": "Microservice Architecture" 
    }
)
print("-"*100)


print("\n\n","="*40,"Response","="*40)

response = update_post_category(patch_request, post_category_id=6)
pprint(response.status_code)
pprint(response.data)
print("="*100)

---------------------------------------- Request ----------------------------------------
<WSGIRequest: PATCH '/post-category/6'>
<rest_framework.request.Request: PATCH '/post-category/6'>
{'_request': <WSGIRequest: PATCH '/post-category/6'>, 'parsers': [<rest_framework.parsers.JSONParser object at 0x7fecb27cf490>], 'authenticators': (), 'negotiator': <rest_framework.negotiation.DefaultContentNegotiation object at 0x7fecb27cf400>, 'parser_context': {'request': <rest_framework.request.Request: PATCH '/post-category/6'>, 'encoding': 'utf-8'}, '_data': <class 'rest_framework.request.Empty'>, '_files': <class 'rest_framework.request.Empty'>, '_full_data': <class 'rest_framework.request.Empty'>, '_content_type': <class 'rest_framework.request.Empty'>, '_stream': <class 'rest_framework.request.Empty'>} 


JSON data sent in the request:
 {'name': 'Microservice Architecture'} 


Query Params sent in the request:
 <QueryDict: {}> 


--------------------------------------------------------------